In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print
!cp  /content/drive/MyDrive/colab/results/* ./results/


Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@main#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
import pandas as pd
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
prevmodel = "gstock-4b-test"
model = "gstock-6b-test"
tickers = [ 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
prevmodel = "val-5a-test"
model = "val-10a-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
prevmodel = "cry-5a-test"
model = "cry-7a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']

In [7]:
findata.EPOCHS=200
prevmodel = "etf-5b-test"
model = "etf-8b-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
prevmodel = "mcap-5b-test"
model = "mcap-7b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS', 'DELL',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']

In [ ]:
pipeline.IS_VERBOSE = False
findata.G_SCALER="minmax"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df1  = pipeline.runModelCombinedVola(tickers, model, mod, True)


In [10]:
pd.options.display.max_columns = None
# addAllocHL(df, 0.10, 1)
# # df1 = df[df.Gain > 0]
# #df1.sort_values('Alloc', ascending=False).head(50)
# df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index]
# #df.sort_values('Alloc', ascending=False).head(50)
df1

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l
0,ARKF,0.58,0.90,237.23,287.56,28.66,28.18,-0.02,0.43,0.69,94.29,191.85,30.33,0.06,0.35,0.71,182.40,242.80,24.45,-0.15
1,ARKK,1.45,0.95,527.22,732.42,45.42,42.29,-0.07,1.10,0.68,320.42,529.69,45.33,-0.00,0.95,0.70,482.24,676.00,40.89,-0.10
2,ARKW,1.48,0.94,639.98,887.53,80.11,76.17,-0.05,1.33,0.72,431.57,651.28,81.95,0.02,1.18,0.64,515.05,777.04,64.91,-0.19
3,CIBR,0.46,0.93,214.14,188.60,59.14,55.91,-0.05,0.33,0.73,231.73,124.82,58.53,-0.01,0.28,0.70,243.19,159.81,52.94,-0.10
4,DAPP,0.39,0.97,144.84,164.36,11.30,11.35,0.00,0.32,0.73,81.85,143.64,12.41,0.10,0.27,0.69,82.24,159.18,8.66,-0.23
5,DIA,1.94,0.93,1093.80,735.48,412.89,393.82,-0.05,1.42,0.82,1310.40,563.90,407.61,-0.01,1.47,0.58,766.23,154.79,379.19,-0.08
6,DTEC,0.38,0.95,204.33,195.44,43.04,41.91,-0.03,0.22,0.73,174.08,115.08,46.70,0.09,0.20,0.68,170.54,174.53,40.77,-0.05
7,EEM,0.33,0.91,102.01,158.25,43.53,41.49,-0.05,0.20,0.66,92.06,100.32,46.19,0.06,0.24,0.72,127.30,179.57,39.13,-0.10
8,FPX,1.02,0.91,469.35,466.41,103.19,104.47,0.01,0.64,0.81,482.24,477.15,109.32,0.06,0.52,0.71,460.98,529.22,97.99,-0.05
9,ICLN,0.26,0.91,118.39,126.77,14.35,14.29,-0.00,0.21,0.72,104.77,110.81,14.81,0.03,0.22,0.79,156.33,179.75,13.59,-0.05


In [11]:
perf_cols = ['Error','Accu','Buy', 'Sell','Error_h', 'Accu_h', 'Buy_h', 'Sell_h','Error_l', 'Accu_l', 'Buy_l','Sell_l']
mean_values  = df1[perf_cols].mean()
mean_df_minmax = pd.DataFrame(mean_values).transpose()
mean_df_minmax

,Error,Accu,Buy,Sell,Error_h,Accu_h,Buy_h,Sell_h,Error_l,Accu_l,Buy_l,Sell_l
0,1.027708,0.939375,555.729792,431.63625,0.755625,0.758958,501.129583,360.026667,0.700833,0.674792,468.506458,360.379792


In [12]:
pipeline.IS_VERBOSE = False
findata.G_SCALER="standard"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df2  = pipeline.runModelCombinedVola(tickers, model, mod, True)


Epoch 1/200
553/557 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0932 - mean_absolute_error: 0.3360
Epoch 1: val_loss improved from inf to 0.06312, saving model to results/etf-8b-test-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
557/557 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 0.0931 - mean_absolute_error: 0.3358 - val_loss: 0.0631 - val_mean_absolute_error: 0.2745
Epoch 2/200
552/557 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0691 - mean_absolute_error: 0.2873
Epoch 2: val_loss improved from 0.06312 to 0.05175, saving model to results/etf-8b-test-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
557/557 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0690 - mean_absolute_error: 0.2872 - val_loss: 0.0518 - val_mean_absolute_error: 0.2471
Epoch 3/200
556/557 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0530 - mean_absolute_e

In [13]:
# addAllocHL(df2, 0.10, 1)
# df2.iloc[(df2.Alloc * abs(df2.Gain_f)).sort_values(ascending=False).index]
df2

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l
0,ARKF,0.53,0.93,245.69,254.86,28.66,25.38,-0.11,0.34,0.74,148.09,203.12,29.15,0.02,0.35,0.71,305.99,297.65,20.52,-0.28
1,ARKK,1.36,0.94,436.82,835.33,45.42,39.63,-0.13,0.92,0.67,193.46,443.56,47.98,0.06,0.77,0.72,471.45,597.51,38.64,-0.15
2,ARKW,1.38,0.93,638.10,971.47,80.11,70.03,-0.13,0.84,0.73,448.37,721.03,79.96,-0.00,0.92,0.68,609.14,699.52,66.02,-0.18
3,CIBR,0.45,0.94,252.63,204.44,59.14,58.57,-0.01,0.21,0.82,282.88,205.02,61.26,0.04,0.21,0.66,265.87,151.48,55.58,-0.06
4,DAPP,0.33,0.97,98.91,177.60,11.30,9.41,-0.17,0.21,0.78,157.63,164.96,12.79,0.13,0.21,0.74,113.63,145.53,10.30,-0.09
5,DIA,1.65,0.97,1226.93,734.19,412.89,403.02,-0.02,0.97,0.79,1196.01,643.92,413.59,0.00,1.03,0.65,858.78,434.02,389.75,-0.06
6,DTEC,0.31,0.95,162.04,200.16,43.04,40.93,-0.05,0.15,0.76,133.00,180.01,43.25,0.01,0.21,0.72,194.05,162.93,39.29,-0.09
7,EEM,0.33,0.93,133.01,129.75,43.53,42.34,-0.03,0.16,0.70,119.40,118.41,44.62,0.02,0.18,0.72,153.90,161.72,39.45,-0.09
8,FPX,1.03,0.94,411.90,563.13,103.19,97.19,-0.06,0.50,0.82,473.21,552.52,103.16,-0.00,0.56,0.78,504.23,623.04,93.70,-0.09
9,ICLN,0.28,0.93,116.76,151.94,14.35,15.04,0.05,0.14,0.73,99.81,128.79,15.15,0.06,0.15,0.74,103.99,156.34,13.57,-0.05


In [15]:
mean_values  = df2[perf_cols].mean()
mean_df_standard = pd.DataFrame(mean_values).transpose()
mean_df_standard - mean_df_minmax

,Error,Accu,Buy,Sell,Error_h,Accu_h,Buy_h,Sell_h,Error_l,Accu_l,Buy_l,Sell_l
0,-0.04375,0.00125,-10.235208,14.970625,-0.251042,0.017292,42.56375,22.926875,-0.1375,0.024792,45.87625,35.921042


In [ ]:
pipeline.IS_VERBOSE = False
findata.G_SCALER="minmax"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj(),
                         pipeline.ValueChange()
                         ]))


df3  = pipeline.runModelCombinedRR(tickers, model, mod, True)


In [ ]:
addAllocHL(df3, 0.10, 1)
df3.iloc[(df3.Alloc * abs(df2.Gain_f)).sort_values(ascending=False).index]

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
4,DAPP,0.86,0.92,119.58,141.49,11.32,12.83,0.13,0.77,0.68,84.25,93.74,13.06,0.15,0.65,0.70,45.39,142.47,10.85,-0.04,0.13,8.584615
24,SMH,4.20,0.95,1169.93,733.57,244.58,257.55,0.05,3.21,0.78,1020.95,322.94,232.98,-0.05,3.07,0.58,378.47,740.49,224.30,-0.08,-0.05,8.500000
31,VUG,2.94,0.98,1455.79,813.20,376.69,388.56,0.03,1.87,0.83,1449.20,382.68,365.46,-0.03,2.07,0.67,774.37,919.94,363.83,-0.03,-0.03,9.133333
2,ARKW,2.87,0.95,669.95,820.08,80.54,91.97,0.14,2.20,0.75,770.68,425.28,89.88,0.12,2.23,0.64,216.82,838.96,79.65,-0.01,0.12,9.083333
19,MGK,2.68,0.97,1260.02,846.53,315.86,324.35,0.03,1.65,0.84,1260.48,360.03,306.82,-0.03,2.03,0.69,769.89,802.47,303.08,-0.04,-0.03,8.700000
1,ARKK,2.54,0.92,442.42,783.33,45.48,50.86,0.12,2.00,0.67,435.13,478.62,52.78,0.16,2.10,0.76,260.83,844.30,45.63,0.00,0.12,8.533333
39,XLK,2.17,0.94,955.14,503.04,222.32,230.74,0.04,1.52,0.84,865.85,267.86,211.10,-0.05,1.73,0.66,633.41,453.02,209.01,-0.06,-0.05,8.200000
13,IXN,0.78,0.94,290.74,193.27,81.99,84.89,0.04,0.48,0.84,288.75,79.37,78.08,-0.05,0.55,0.60,132.70,206.58,77.35,-0.06,-0.05,8.200000
18,JETS,0.76,0.96,136.82,125.43,18.39,16.93,-0.08,0.66,0.66,105.29,35.72,18.66,0.01,0.74,0.71,54.99,122.44,17.78,-0.03,-0.03,8.266667
23,QQQ,4.65,0.96,1976.63,1081.00,476.65,499.59,0.05,3.18,0.73,1816.14,292.39,463.02,-0.03,2.98,0.66,854.08,1082.39,457.97,-0.04,-0.03,8.266667


In [ ]:
mean_values3  = df3[perf_cols].mean()
mean_df_rr_mimmax = pd.DataFrame(mean_values3).transpose()
mean_df_rr_mimmax - mean_df_minmax

,Error,Accu,Buy,Sell,Error_h,Accu_h,Buy_h,Sell_h,Error_l,Accu_l,Buy_l,Sell_l
0,-0.567083,0.001875,0.177292,-14.30125,-0.278125,-0.025,10.749167,-215.529583,-0.546042,-0.018958,-227.783125,34.535208


In [ ]:
pipeline.IS_VERBOSE = False
findata.G_SCALER="standard"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj(),
                         pipeline.ValueChange()
                         ]))


df4  = pipeline.runModelCombinedRR(tickers, model, mod, True)


In [ ]:
df4

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l
0,ARKF,-1.31,0.96,184.85,268.59,28.66,31.39,0.10,-1.10,0.74,197.42,164.88,32.69,0.14,-1.19,0.73,106.21,292.30,30.43,0.06
1,ARKK,-0.95,0.96,428.82,804.33,45.48,55.33,0.22,-0.96,0.73,463.78,410.40,55.01,0.21,-0.81,0.73,246.78,671.15,47.93,0.05
2,ARKW,-1.74,0.97,714.57,853.06,80.54,88.82,0.10,-1.75,0.70,641.51,452.53,90.08,0.12,-1.91,0.73,434.55,728.45,88.69,0.10
3,CIBR,0.87,0.95,252.68,180.88,59.13,60.61,0.03,0.86,0.73,214.12,60.98,57.11,-0.03,0.81,0.67,117.90,152.34,57.83,-0.02
4,DAPP,-0.84,0.95,133.34,143.37,11.32,12.29,0.09,-0.81,0.69,106.02,58.74,12.54,0.11,-0.80,0.71,49.40,168.53,10.10,-0.11
5,DIA,0.40,0.98,1104.35,726.38,412.77,408.11,-0.01,0.37,0.80,1078.80,319.30,400.61,-0.03,0.35,0.71,710.87,509.53,411.27,-0.00
6,DTEC,2.91,0.97,198.78,213.91,43.10,43.17,0.00,2.69,0.80,217.66,72.75,44.31,0.03,2.49,0.71,109.63,150.46,44.27,0.03
7,EEM,7.65,0.91,119.15,140.40,43.55,42.18,-0.03,7.28,0.63,115.43,42.02,42.88,-0.02,6.35,0.68,43.06,159.51,45.07,0.03
8,FPX,4.57,0.96,432.18,605.08,103.31,101.00,-0.02,4.11,0.78,385.84,237.98,99.47,-0.04,4.45,0.70,229.16,444.38,106.58,0.03
9,ICLN,-4.84,0.95,76.01,163.35,14.38,15.07,0.05,-4.81,0.73,95.28,66.80,15.13,0.05,-5.35,0.68,48.02,122.80,15.06,0.05


In [ ]:
mean_values4  = df4[perf_cols].mean()
mean_df_rr_standard = pd.DataFrame(mean_values4).transpose()
mean_df_rr_standard - mean_df_minmax

,Error,Accu,Buy,Sell,Error_h,Accu_h,Buy_h,Sell_h,Error_l,Accu_l,Buy_l,Sell_l
0,-2.355417,0.022917,-9.607292,-0.67875,-0.759583,-0.005,14.369167,-177.270417,-7.873125,0.016875,-193.345208,16.612708


In [ ]:
findata.EPOCHS=200
model = "vols-6b-test"
pending = ['ASTR', 'SMWB', 'CZOO', 'DMTK']
next = []
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DPSI', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-6a-test"
next = []
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-8a-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-8a-test"
tickers = [ 'FXB', 'FXE', 'FXY', 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
dfprev  = pipeline.runModelCombinedVola(tickers, prevmodel, mod, False, loss=lossfn)

In [ ]:
addAllocHL(dfprev, 0.10, 1)
df1prev = dfprev[dfprev.Gain > 0]
pd.options.display.max_columns = None
# df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
df1prev.sort_values('Alloc', ascending=False).head(50)

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=findata.Huber())

In [ ]:
addAllocHL(df, 0.10, 1)
df1 = df[df.Gain > 0]
pd.options.display.max_columns = None
# df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
df1.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
3,AMBA,2.85,0.92,862.61,838.13,47.57,58.03,0.22,1.85,0.65,582.85,728.60,59.68,0.25,1.52,0.66,736.82,680.75,41.51,-0.13,0.22,8.836364
31,HIMS,0.40,0.94,162.71,112.11,15.58,17.21,0.10,0.21,0.73,157.26,104.03,18.13,0.16,0.21,0.71,92.81,75.66,13.92,-0.11,0.10,8.800000
26,FIVE,3.83,0.92,1028.27,1246.17,76.96,87.05,0.13,2.29,0.77,851.42,977.30,84.90,0.10,2.12,0.79,1215.69,1429.93,72.89,-0.05,0.10,8.400000
37,JKS,1.46,0.92,409.97,440.01,19.49,21.42,0.10,0.88,0.55,175.43,341.74,22.55,0.16,0.69,0.78,450.51,473.02,17.37,-0.11,0.10,8.400000
8,CELH,1.37,0.91,489.15,437.75,40.17,44.63,0.11,0.82,0.75,455.09,338.20,45.65,0.14,0.84,0.72,411.00,317.84,37.28,-0.07,0.11,8.281818
71,TRIP,0.64,0.88,236.00,240.08,13.80,17.04,0.23,0.50,0.74,281.51,280.49,19.05,0.38,0.42,0.77,166.51,197.88,13.84,0.00,0.23,8.278261
30,GLBE,1.22,0.93,242.73,369.42,34.56,36.66,0.06,0.63,0.74,265.35,425.20,38.76,0.12,0.64,0.76,324.16,390.45,30.55,-0.12,0.06,8.133333
21,DXCM,2.14,0.93,665.03,985.19,74.65,78.28,0.05,1.24,0.73,750.73,874.77,88.73,0.19,1.19,0.69,643.37,662.28,70.28,-0.06,0.05,7.900000
16,DDOG,2.37,0.95,977.57,946.48,114.98,118.75,0.03,1.44,0.73,1233.50,863.84,122.97,0.07,1.44,0.78,875.59,1125.96,108.78,-0.05,0.03,7.833333
10,CHGG,1.22,0.84,253.02,485.79,2.05,2.59,0.26,0.89,0.64,224.54,323.34,3.19,0.56,0.78,0.73,207.16,248.89,1.93,-0.06,0.26,7.784615


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [16]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/

sending incremental file list
etf-8b-test-adjclose-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-8b-test-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-8b-test-high-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-8b-test-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-8b-test-low-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-8b-test-low-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras

sent 57,677,497 bytes  received 130 bytes  115,355,254.00 bytes/sec
total size is 467,998,611  speedup is 8.11
